In [1]:
import json
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm.keras import TQDMNotebookCallback  # Add this for tqdm progress bar

# Load JSON files
def load_json(file_path):
    with open(file_path) as f:
        return json.load(f)

creatures = load_json('data/creatures.json')
great_old_ones = load_json('data/great_old_ones.json')
lesser_old_ones = load_json('data/lesser_old_ones.json')
outer_gods = load_json('data/outer_gods.json')
races = load_json('data/races.json')

# Extract names from JSON
def extract_names(data):
    return [item['name'] for item in data]

json_names = (
    extract_names(creatures) +
    extract_names(great_old_ones) +
    extract_names(lesser_old_ones) +
    extract_names(outer_gods) +
    [race['race'] for race in races]
)

# Load Lovecraft fiction data
lovecraft_data = pd.read_csv('data/lovecraft_fiction.csv')

# Clean text function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s\']', '', text)  # Allow apostrophes
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Apply cleaning to the 'Text' column
lovecraft_data['Cleaned_Text'] = lovecraft_data['Text'].apply(clean_text)

# Filter texts based on names
def filter_texts(data_frame, names):
    return [
        text for text in data_frame['Text']
        if any(name.lower() in text.lower() for name in names)
    ]

filtered_texts = filter_texts(lovecraft_data, json_names)

# Count occurrences of names in filtered texts
name_counts = Counter()
for text in filtered_texts:
    for name in json_names:
        name_counts[name] += text.lower().count(name.lower())

# Create DataFrame for name counts
name_counts_df = pd.DataFrame(name_counts.items(), columns=['Name', 'Count'])

# Plot top names
def plot_top_names(name_counts_df, top_n=20):
    top_names = name_counts_df.nlargest(top_n, 'Count')
    plt.figure(figsize=(12, 6))
    plt.bar(top_names['Name'], top_names['Count'], color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.title('Top Names in Lovecraft’s Works')
    plt.xlabel('Name')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()

plot_top_names(name_counts_df)

# Save the name counts
name_counts_df.to_csv('data_processed/lovecraft_name_counts.csv', index=False)

# Tokenize descriptions and count tokens
stop_words = set(stopwords.words('english'))

def tokenize_descriptions(lovecraft_data):
    entities = []
    for _, row in lovecraft_data.iterrows():
        title = row['Title'].lower()
        text = row['Cleaned_Text']
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in stop_words]
        sentences = sent_tokenize(text)
        phrases = [phrase for phrase in sentences if len(phrase.split()) > 1]
        entities.append({'name': title, 'description': text, 'tokens': tokens, 'phrases': phrases})
    return pd.DataFrame(entities)

processed_df = tokenize_descriptions(lovecraft_data)
processed_df.to_csv('data_processed/lovecraft_processed_entities.csv', index=False)

# Count token frequencies
all_tokens = processed_df['tokens'].explode().tolist()
token_counts = Counter(all_tokens)
token_counts_df = pd.DataFrame(token_counts.items(), columns=['Token', 'Frequency'])
token_counts_df.to_csv('data_processed/lovecraft_token_frequencies.csv', index=False)

# Extract unique phrases
all_phrases = [phrase for phrases in processed_df['phrases'] for phrase in phrases]
phrase_counts = Counter(all_phrases)
phrase_counts_df = pd.DataFrame(phrase_counts.items(), columns=['Phrase', 'Frequency'])
phrase_counts_df.to_csv('data_processed/lovecraft_phrase_frequencies.csv', index=False)

# Save final name list combining JSON names with filtered text names
combined_names = list(set([name.lower() for name in json_names] + list(token_counts.keys())))
with open('data_processed/final_name_list.txt', 'w') as f:
    for name in combined_names:
        f.write(f"{name}\n")

# Count occurrences of names
name_occurrences = Counter()
for name in json_names:
    name_occurrences[name] = name_occurrences.get(name, 0) + sum(row['Cleaned_Text'].lower().count(name.lower()) for _, row in lovecraft_data.iterrows())

# Count occurrences of phrases
phrase_occurrences = Counter()
for phrases in processed_df['phrases']:
    phrase_occurrences.update(phrases)

# Create DataFrames for name counts and phrase counts
name_counts_df = pd.DataFrame(name_occurrences.items(), columns=['Name', 'Count'])
phrase_counts_df = pd.DataFrame(phrase_occurrences.items(), columns=['Phrase', 'Frequency'])

# Get top 50 names and phrases
top_names = name_counts_df.nlargest(50, 'Count')
top_phrases = phrase_counts_df.nlargest(50, 'Frequency')

# Print or save the top names and phrases
top_names.to_csv('data_processed/top_50_names.csv', index=False)
top_phrases.to_csv('data_processed/top_50_phrases.csv', index=False)

# Tokenization for deep learning model
tokenizer = Tokenizer(num_words=10000)  # Limit vocab size
tokenizer.fit_on_texts(lovecraft_data['Cleaned_Text'].tolist())

# Convert texts to sequences and pad them
sequences = tokenizer.texts_to_sequences(lovecraft_data['Cleaned_Text'].tolist())
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

# Prepare input (X) and output (y)
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

# Use sparse categorical crossentropy instead of one-hot encoding
y_sparse = y  # No need to convert to categorical

# Define and train LSTM model with sparse categorical crossentropy
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X.shape[1]),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(10000, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Add tqdm progress bar callback to monitor training progress
model.fit(X, y_sparse, batch_size=64, epochs=20, validation_split=0.2, callbacks=[TQDMNotebookCallback()])

# Save the model
model.save('lovecraft_lstm_model.h5')

print("Model training completed!")


c:\Users\Marc Jay\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'TQDMNotebookCallback' from 'tqdm.keras' (c:\Users\Marc Jay\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\keras.py)